<a href="https://colab.research.google.com/github/devshah21/nuclei_segmentation/blob/main/Copy_of_Copy_of_understanding_semantic_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from zipfile import ZipFile

In [8]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
with ZipFile('gdrive/MyDrive/google_colab_files/stage1_train.zip', 'r') as zipObj:
  zipObj.extractall('gdrive/MyDrive/google_colab_files/traindata')

In [7]:
with ZipFile('gdrive/MyDrive/google_colab_files/stage1_test.zip', 'r') as zipObj:
  zipObj.extractall('gdrive/MyDrive/google_colab_files/testdata')

In [25]:
traindatapath = '/content/drive/MyDrive/google_colab_files/traindata'
testdatapath = '/content/gdrive/MyDrive/google_colab_files/testdata'

In [26]:
import os

In [28]:
trainids = next(os.walk(traindatapath))[1]

In [31]:
try:
    testids = next(os.walk(testdatapath))[1]
except StopIteration:
    testids = []

In [32]:
imgwidth = 128
imgheight = 128
channels = 3

In [33]:
import numpy as np

xtrain = np.zeros((len(trainids), imgheight, imgwidth, channels), dtype=np.uint8)
ytrain = np.zeros((len(trainids), imgheight, imgwidth, 1), dtype=np.bool)

<ipython-input-33-119cd78b6827>:4: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  ytrain = np.zeros((len(trainids), imgheight, imgwidth, 1), dtype=np.bool)


In [34]:
import random
from skimage.io import imread, imshow
from skimage.transform import resize
import matplotlib.pyplot as plt
import tensorflow as tf

Define the UNET Architecture now








In [ ]:
inputs = tf.keras.layers.Input((imgwidth, imgheight, channels))

In [ ]:
# Convert the integer values for the model into float values by dividing by 255. 
values = tf.keras.layers.Lambda(lambda x: x/255)(inputs)

# Define the first conv layer, input the parameters from the U-Net diagram
conv1 = tf.keras.layers.Conv2D(16, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(values)
# Prevent overfitting by using dropout function
conv1 = tf.keras.layers.Dropout(0.1)(conv1)
conv1 = tf.keras.layers.Conv2D(16, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(conv1)
down1 = tf.keras.layers.MaxPooling2D((2,2))(conv1)

Repeat the following paramaters for the rest of the UNET model. The main focus here is to just define the structure/architecture for UNET.

In [ ]:
conv2 = tf.keras.layers.Conv2D(32, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(down1)
conv2 = tf.keras.layers.Dropout(0.1)(conv2)
conv2 = tf.keras.layers.Conv2D(32, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(conv2)
down2 = tf.keras.layers.MaxPooling2D((2,2))(conv2)

In [ ]:
conv3 = tf.keras.layers.Conv2D(64, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(down2)
conv3 = tf.keras.layers.Dropout(0.2)(conv3)
conv3 = tf.keras.layers.Conv2D(64, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(conv3)
down3 = tf.keras.layers.MaxPooling2D((2,2))(conv3)

In [ ]:
conv4 = tf.keras.layers.Conv2D(128, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(down3)
conv4 = tf.keras.layers.Dropout(0.2)(conv4)
conv4 = tf.keras.layers.Conv2D(128, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(conv4)
down4 = tf.keras.layers.MaxPooling2D(pool_size =(2,2))(conv4)

In [ ]:
conv5 = tf.keras.layers.Conv2D(256, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(down4)
conv5 = tf.keras.layers.Dropout(0.3)(conv5)
conv5 = tf.keras.layers.Conv2D(256, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(conv5)

Now we've finished the first half of the UNET architecture, the next step is to expand the model out after contracting it.

In [ ]:
up6 = tf.keras.layers.Conv2DTranspose(128, (2,2), strides = (2,2), padding = 'same')(conv5)
up6 = tf.keras.layers.concatenate([up6, conv4]) #skip connections
conv6 = tf.keras.layers.Conv2D(128, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(up6)
conv6 = tf.keras.layers.Dropout(0.2)(conv6)
conv6 = tf.keras.layers.Conv2D(128, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(conv6)

In [ ]:
up7 = tf.keras.layers.Conv2DTranspose(64, (2,2), strides = (2,2), padding = 'same')(conv6)
up7 = tf.keras.layers.concatenate([up7, conv3]) #skip connections
conv7 = tf.keras.layers.Conv2D(64, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(up7)
conv7 = tf.keras.layers.Dropout(0.2)(conv7)
conv7 = tf.keras.layers.Conv2D(64, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(conv7)

In [ ]:
up8 = tf.keras.layers.Conv2DTranspose(32, (2,2), strides = (2,2), padding = 'same')(conv7)
up8 = tf.keras.layers.concatenate([up8, conv2])
conv8 = tf.keras.layers.Conv2D(32, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(up8)
conv8 = tf.keras.layers.Dropout(0.1)(conv8)
conv8 = tf.keras.layers.Conv2D(32, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(conv8) 

In [ ]:
up9 = tf.keras.layers.Conv2DTranspose(16, (2,2), strides = (2,2), padding = 'same')(conv8)
up9 = tf.keras.layers.concatenate([up9, conv1], axis=3)
conv9 = tf.keras.layers.Conv2D(16, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(up9)
conv9 = tf.keras.layers.Dropout(0.1)(conv9)
conv9 = tf.keras.layers.Conv2D(16, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(conv9) 

In [ ]:
outputs = tf.keras.layers.Conv2D(1, (1,1), activation = 'sigmoid')(conv9)

In [ ]:
model = tf.keras.Model(inputs=[inputs], outputs = [outputs])
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics =['accuracy'])